# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 60 new papers today
          26 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/25 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2211.07653


extracting tarball to tmp_2211.07653...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.07656


/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2211.07653/natnotes.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.08356


/tmp/ipykernel_2264/4030337529.py:34: LatexWarning: 2211.08356 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2211.08388


extracting tarball to tmp_2211.08388...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.07659-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.07659) | **Feedback reshapes the baryon distribution within haloes, in halo  outskirts, and beyond: the closure radius from dwarfs to massive clusters**  |
|| Mohammadreza Ayromlou, Dylan Nelson, <mark>Annalisa Pillepich</mark> |
|*Appeared on*| *2022-11-16*|
|*Comments*| *Submitted to MNRAS*|
|**Abstract**| We explore three sets of cosmological hydrodynamical simulations, IllustrisTNG, EAGLE, and SIMBA, to investigate the physical processes impacting the distribution of baryons in and around haloes across an unprecedented mass range of $10^8<M_{\rm 200c}/{\rm M_{\odot}}<10^{15}$, from the halo centre out to scales as large as $30\,R_{\rm 200c}$. We demonstrate that baryonic feedback mechanisms significantly redistribute gas, lowering the baryon fractions inside haloes while simultaneously accumulating this material outside the virial radius. To understand this large-scale baryonic redistribution and identify the dominant physical processes responsible, we examine several variants of TNG that selectively exclude stellar and AGN feedback, cooling, and radiation. We find that heating from the UV background in low-mass haloes, stellar feedback in intermediate-mass haloes, and AGN feedback in groups ($10^{12} \leq M_{\rm 200c}/{\rm M_{\odot}}<10^{14}$) are the dominant processes. Galaxy clusters are the least influenced by these processes on large scales. We introduce a new halo mass-dependent characteristic scale, the closure radius $R_{\rm c}$, within which all baryons associated with haloes are found. For groups and clusters, we introduce a universal relation between this scale and the halo baryon fraction: $R_{\rm c}/R_{\rm 200c,500c}-1=\beta(z)(1-f_{\rm b}(<R_{\rm 200c,500c})/f_{\rm b,cosmic})$, where $\beta(z)=\alpha\,(1+z)^\gamma$, and $\alpha$ and $\gamma$ are free parameters fit using the simulations. Accordingly, we predict that all baryons associated with observed X-ray haloes can be found within $R_{\rm c}\sim 1.5-2.5 R_{\rm 200c}$. Our results can be used to constrain theoretical models, particularly the physics of supernova and AGN feedback, as well as their interplay with environmental processes, through comparison with current and future X-ray and SZ observations. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.07667-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.07667) | **The First 3$π$ 3D Map of ISM Dust Temperature**  |
|| Ioana A. Zelko, Douglas P. Finkbeiner, <mark>Albert Lee</mark>, <mark>Gregory Green</mark> |
|*Appeared on*| *2022-11-16*|
|*Comments*| *11 Figures, 19 pages*|
|**Abstract**| We present the first large-scale 3D map of interstellar dust temperature. We build upon existing 3D reddening maps derived from starlight absorption (Bayestar19), covering 3/4 of the sky. Starting with the column density for each of 500 million 3D voxels, we propose a temperature and emissivity power-law slope ($\beta$) for each of them, and integrate along the line of sight to synthesize an emission map in five frequency bands observed by \emph{Planck} and \emph{IRAS}. The reconstructed emission map is constrained to match observations on a $10'$ scale, and does so with good fidelity. We produce 3D temperature maps at resolutions of $110', 55', $and $27'$. We assess performance on Cepheus, a dust cloud with two distinct components along the line of sight, and find distinct temperatures for the two components. We thus show that this methodology has enough precision to constrain clouds with different temperature along the line of sight up to $1-\sigma$ error. This would be an important result for dust frequency decorrelation foreground analysis for cosmic microwave background experiments, which would be impacted by a line-of-sight with varying temperature and magnetic field components. In addition to $T$ and $\beta$, we constrain the conversion factor between emission optical depth and reddening. This conversion factor is assumed to be constant in commonly used emission-based reddening maps. However, this work shows a factor of two variation that may prove significant for some applications. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.08294-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.08294) | **Radial Velocity Survey for Planets around Young stars (RVSPY) A  transiting warm super-Jovian planet around HD 114082, a young star with a  debris disk**  |
|| <mark>O. Zakhozhay</mark>, et al. -- incl., <mark>R. Launhardt</mark>, <mark>T. Trifonov</mark>, <mark>M. Kürster</mark>, <mark>Th. Henning</mark> |
|*Appeared on*| *2022-11-16*|
|*Comments*| *10 pages, 9 figures, 5 tables; Accepted for publication in A&A Letters*|
|**Abstract**| Aiming to detect planetary companions to young stars with debris disks via the radial velocity method, we observed HD114082 during April 2018 - August 2022 as one of the targets of our RVSPY program (Radial Velocity Survey for Planets around Young stars). We used the FEROS spectrograph, mounted to the MPG/ESO 2.2 m telescope in Chile, to obtain high signal-to-noise spectra and time series of precise radial velocities (RVs). Additionally, we analyzed archival HARPS spectra and TESS photometric data. We used the CERES, CERES++ and SERVAL pipelines to derive RVs and activity indicators and ExoStriker for the independent and combined analysis of the RVs and TESS photometry. We report the discovery of a warm super-Jovian companion around HD114082 based on a 109.8$\pm$0.4 day signal in the combined RV data from FEROS and HARPS, and on one transit event in the TESS photometry. The best-fit model indicates a 8.0$\pm$1.0 Mjup companion with a radius of 1.00$\pm$0.03 Rjup in an orbit with a semi-major axis of 0.51$\pm$0.01 au and an eccentricity of 0.4$\pm$0.04. The companions orbit is in agreement with the known near edge-on debris disk located at about 28 au. HD114082b is possibly the youngest (15$\pm$6 Myr), and one of only three younger than 100 Myr giant planetary companions for which both their mass and radius have been determined observationally. It is probably the first properly model-constraining giant planet that allows distinguishing between hot and cold-start models. It is significantly more compatible with the cold-start model. |

## Failed papers

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.08255-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.08255) | **EIGER II. first spectroscopic characterisation of the young stars and  ionised gas associated with strong H$β$ and [OIII] line-emission in  galaxies at z=5-7 with JWST**  |
|| Jorryt Matthee, et al. -- incl., <mark>Simon J. Lilly</mark> |
|*Appeared on*| *2022-11-16*|
|*Comments*| *Submitted to ApJ. Main text 22 pages, 20 figures. Main results in Figs 14 (Xi_ion), 15 (MEx diagram),17 (MZR), 19 ([OIII] luminosity density). Comments welcome*|
|**Abstract**| We present emission-line measurements and physical interpretations for a sample of 117 [OIII] emitting galaxies at $z=5.33-6.93$, using the first deep JWST/NIRCam wide field slitless spectroscopic observations. Our 9.7-hour integration is centered upon the $z=6.3$ quasar J0100+2802 -- the first of six fields targeted by the EIGER survey -- and covers $\lambda=3-4$ microns. We detect 133 [OIII] doublets, but merge pairs within $\approx$10 kpc and 600 km s$^{-1}$, motivated by their small scale clustering excess. We detect H$\beta$ in 68 and H$\gamma$ in two galaxies. The galaxies are characterised by a UV luminosity M$_{\rm UV}\sim-19.7$ ($-17.7$ to $-22.3$), mass ~$10^8$ $(10^{6.8-9.8})$ M$_{\odot}$, H$\beta$ and [OIII] EWs $\approx$ 1000 Angstrom (up to 3000 Angstrom), young ages (~5 Myr), a highly excited interstellar medium ([OIII]/H$\beta\approx6$) and low dust attenuations. These high EWs are very rare in the local Universe, but we show they are ubiquitous at $z\sim6$ based on the measured number densities. The stacked spectrum reveals H$\gamma$ and [OIII]$_{4364}$ which shows that the galaxies are typically dust and metal poor (E(B-V)=0.1, 12+log(O/H)=7.4) with a high electron temperature ($2\times10^4$ K) and a high production efficiency of ionising photons ($\xi_{\rm ion}=10^{25.6}$ Hz erg$^{-1}$). We further show the existence of a strong mass-metallicity relation as expected by simulations. The young highly ionising stellar populations, moderately low metallicities, low dust attenuations and high ionisation state in z~6 galaxies conspire to maximise the [OIII] output from galaxies, yielding an [OIII] luminosity density at z~6 that is significantly higher than at z~2, despite the order of magnitude decline in cosmic star formation. Thus, [OIII] emission-line surveys with JWST prove a highly efficient method to trace the galaxy density in the epoch of reionization. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.08348-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.08348) | **Non-perturbative non-Gaussianity and primordial black holes**  |
|| Andrew D. Gow, et al. -- incl., <mark>Joseph H. P. Jackson</mark> |
|*Appeared on*| *2022-11-16*|
|*Comments*| *8 pages, 2 figures*|
|**Abstract**| We present a non-perturbative method for calculating the abundance of primordial black holes given an arbitrary one-point probability distribution function for the primordial curvature perturbation, $P(\zeta)$. A non-perturbative method is essential when considering non-Gaussianities that cannot be treated using a conventional perturbative expansion. To determine the full statistics of the density field, we relate $\zeta$ to a Gaussian field by equating the cumulative distribution functions. We consider two examples: a specific local-type non-Gaussian distribution arising from ultra slow roll models, and a general piecewise model for $P(\zeta)$ with an exponential tail. We demonstrate that the enhancement of primordial black hole formation is due to the intermediate regime, rather than the far tail. We also show that non-Gaussianity can have a significant impact on the shape of the primordial black hole mass distribution. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.08355-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.08355) | **Galaxy And Mass Assembly: Galaxy Morphology in the Green Valley,  Prominent rings and looser Spiral Arms**  |
|| <mark>Dominic Smith</mark>, et al. -- incl., <mark>Christopher P. A. Henry</mark>, <mark>Jochen Liske</mark>, <mark>Lee S. Kelvin</mark> |
|*Appeared on*| *2022-11-16*|
|*Comments*| *11 pages, 21 figures, accepted to MNRAS*|
|**Abstract**| Galaxies broadly fall into two categories: star-forming (blue) galaxies and quiescent (red) galaxies. In between, one finds the less populated ``green valley". Some of these galaxies are suspected to be in the process of ceasing their star-formation through a gradual exhaustion of gas supply or already dead and are experiencing a rejuvenation of star-formation through fuel injection. We use the Galaxy And Mass Assembly database and the Galaxy Zoo citizen science morphological estimates to compare the morphology of galaxies in the green valley against those in the red sequence and blue cloud. Our goal is to examine the structural differences within galaxies that fall in the green valley, and what brings them there. Previous results found disc features such as rings and lenses are more prominently represented in the green valley population. We revisit this with a similar sized data set of galaxies with morphology labels provided by the Galaxy Zoo for the GAMA fields based on new KiDS images. Our aim is to compare qualitatively the results from expert classification to that of citizen science. We observe that ring structures are indeed found more commonly in green valley galaxies compared to their red and blue counterparts. We suggest that ring structures are a consequence of disc galaxies in the green valley actively exhibiting characteristics of fading discs and evolving disc morphology of galaxies. We note that the progression from blue to red correlates with loosening spiral arm structure. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.07836-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.07836) | **Evolution of Gas, and Star Formation from z = 0 to 5**  |
|| Nick Scoville, et al. -- incl., <mark>Henry McCracken</mark>, <mark>Herve Aussel</mark> |
|*Appeared on*| *2022-11-16*|
|*Comments*| *AAS Russel Lecture June 2022 (N. Scoville), 18 pages. arXiv admin note: text overlap with arXiv:1702.04729*|
|**Abstract**| ALMA observations of the long wavelength dust continuum are used to estimate the gas masses in a sample of 708 star-forming (SF) galaxies at z = 0.3 to 4.5. We determine the dependence of gas masses and star formation efficiencies (SFE=SFR per unit gass mass). We find that 70 percent of the increase in SFRs of the MS is due to the increased gas masses at earlier epochs while 30 percent is due to increased efficiency of SF. For galaxies above the MS this is reversed with 70 percent of the increased SFR relative to the MS being due to elevated SFEs. Thus, the major evolution of star formation activity at early epochs is driven by increased gas masses, while the starburst activity taking galaxies above the MS is due to enhanced triggering of star formation (likely due to galactic merging). The interstellar gas peaks at z = 2 and dominates the stellar mass down to z = 1.2. Accretion rates needed to maintain continuity of the MS evolution exceed 100 Msun per yr at z > 2. The galactic gas contents are likely the driving determinant for both the rise in SF and AGN activity from z = 5 to their peak at z = 2 and subsequent fall to lower z. We suggest that for self-gravitating clouds with supersonic turbulence, cloud collisions and the filamentary structure of the clouds regulate the star formation activity. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'utf-8' codec can't decode byte 0xd5 in position 6001: invalid continuation byte</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.08356-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.08356) | **Monte Carlo Simulation of Lane-Emden Type Equations Arising in  Astrophysics**  |
|| Samah H. El-Essawy, et al. -- incl., <mark>Helal I. Abdel Rahman</mark> |
|*Appeared on*| *2022-11-16*|
|*Comments*| **|
|**Abstract**| Monte Carlo (MC) method played an essential role in many areas of human activity and has found application in many branches of physical science. This paper proposes a computational technique based on MC algorithms to solve Lane-Emden (LE) type equations. We analyze four LE equations arising in astrophysics: the positive and negative indices of the polytropic gas spheres, the isothermal gas sphere, and the white dwarf equation. We calculated eleven models (i.e., eleven LE equations) of the positive index polytropes, nine for the negative index polytrope, the isothermal gas sphere, and the white dwarf equation. Comparing the MC and numerical/analytical models gives good agreement for the four LE equations under study. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in 

 _build/html/2211.07659.md
    + _build/html/tmp_2211.07659/./Figs/R_Closure_TNG_EAGLE_SIMBA.png
    + _build/html/tmp_2211.07659/./Figs/baryonFrac_2dhist_TNG_EAGLE_SIMBA.png
    + _build/html/tmp_2211.07659/./Figs/R_Closure_TNG_VAR.png
    + _build/html/tmp_2211.07659/./Figs/baryonFrac_2dhist_cum_TNG_VAR0000_z0.png
    + _build/html/tmp_2211.07659/./Figs/baryonFrac_2dhist_cum_TNG_VAR2101_z0.png
    + _build/html/tmp_2211.07659/./Figs/baryonFrac_2dhist_cum_TNG_VAR2201_z0.png
    + _build/html/tmp_2211.07659/./Figs/baryonFrac_2dhist_cum_TNG_VAR0030_z0.png
    + _build/html/tmp_2211.07659/./Figs/R_Closure_fit_z0.png
    + _build/html/tmp_2211.07659/./Figs/R_Closure_fit_beta.png
exported in  _build/html/2211.07667.md
    + _build/html/tmp_2211.07667/./figures/smoothing/planck_545.jpg
    + _build/html/tmp_2211.07667/./figures/smoothing/total_zoom_in_reconstructed_emisssion_545.jpg
    + _build/html/tmp_2211.07667/./figures/smoothing/planck_zoom_500_545_smooth_1024.jpg
    + _build/html/tm

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\reza}[1]{\color{red}#1\color{black}}$
$\newcommand{\comment}[1]{\textbf{[#1]}}$
$\newcommand{\rezac}[1]{\color{green}\textbf{[#1] }\color{black}}$
$\newcommand{\eg}[0]{\textnormal{e.g. }}$
$\newcommand{\ie}[0]{\textnormal{i.e. }}$
$\newcommand{\Msun}[0]{ \textnormal{M}_{\textnormal{\astrosun}}}$
$\newcommand{\asun}[0]{_{\textnormal{\astrosun}}}$
$\newcommand{\tn}[1]{\textnormal{#1}}$
$\newcommand{\sub}[1]{_{\textnormal{#1}}}$
$\newcommand{\bs}[1]{\boldsymbol{#1}}$
$\newcommand{\lgal}[0]{\textsc{L-Galaxies}}$
$\newcommand{\kms}{\rm km s^{-1}}$
$\newcommand{\mgii}{Mg \textsc{ii} }$
$\newcommand{\ap}[1]{{\color{magenta} #1}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\reza$}[1]{\color{red}#1\color{black}}$
$\newcommand{$\comment$}[1]{\textbf{[#1]}}$
$\newcommand{$\reza$c}[1]{\color{green}\textbf{[#1] }\color{black}}$
$\newcommand{$\eg$}[0]{\textnormal{e.g. }}$
$\newcommand{$\ie$}[0]{\textnormal{i.e. }}$
$\newcommand{$\Msun$}[0]{ \textnormal{M}_{\textnormal{\astrosun}}}$
$\newcommand{$\asun$}[0]{_{\textnormal{\astrosun}}}$
$\newcommand{$\tn$}[1]{\textnormal{#1}}$
$\newcommand{$\sub$}[1]{_{\textnormal{#1}}}$
$\newcommand{$\bs$}[1]{\boldsymbol{#1}}$
$\newcommand{$\lgal$}[0]{$\textsc${L-Galaxies}}$
$\newcommand{$\kms$}{\rm km s^{-1}}$
$\newcommand{$\mgii$}{Mg $\textsc${ii} }$
$\newcommand{$\ap$}[1]{{\color{magenta} #1}}$</div>



<div id="title">

# Feedback reshapes the baryon distribution within haloes, in halo outskirts, and beyond: the closure radius from dwarfs to massive clusters

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.07659-b31b1b.svg)](https://arxiv.org/abs/2211.07659)<mark>Appeared on: 2022-11-16</mark> - _Submitted to MNRAS_

</div>
<div id="authors">

Mohammadreza Ayromlou, Dylan Nelson, <mark><mark>Annalisa Pillepich</mark></mark>

</div>
<div id="abstract">

**Abstract:** We explore three sets of cosmological hydrodynamical simulations, IllustrisTNG (TNG50, TNG100, TNG300), EAGLE, and SIMBA, to investigate the physical processes impacting the distribution of baryons in and around haloes across an unprecedented mass range of$10^8 < M_{\rm 200c}/{\rm M_{\odot}} < 10^{15}$, from the halo centre out to scales as large as$30 R_{\rm 200c}$. We demonstrate that baryonic feedback mechanisms significantly redistribute gas, lowering the baryon fractions inside haloes while simultaneously accumulating this material outside the virial radius. To understand this large-scale baryonic redistribution and identify the dominant physical processes responsible, we examine several variants of TNG that selectively exclude stellar and AGN feedback, cooling, and radiation. We find that heating from the UV background in low-mass haloes ($10^{8} \leq M_{\rm 200c}/{\rm M_{\odot}}<10^{10}$), stellar feedback in intermediate-mass haloes ($10^{10} \leq M_{\rm 200c}/{\rm M_{\odot}}<10^{12}$), and AGN feedback in groups ($10^{12} \leq M_{\rm 200c}/{\rm M_{\odot}}<10^{14}$) are the dominant processes. Galaxy clusters ($M_{\rm 200c}/{\rm M_{\odot}}>10^{14}$) are the least influenced by these processes on large scales. We introduce a new halo mass-dependent characteristic scale, the closure radius$R_{\rm c}$, within which all baryons associated with haloes are found. For groups and clusters, we introduce a universal relation between this scale and the halo baryon fraction:$R_{\rm c}/R_{\rm 200c,500c} - 1 = \beta(z) (1 - f_{\rm b}(<R_{\rm 200c,500c})/f_{\rm b,cosmic})$, where$\beta(z) = \alpha (1+z)^\gamma$, and$\alpha$and$\gamma$are free parameters fit using the simulations. Accordingly, we predict that all baryons associated with observed X-ray haloes can be found within$R_{\rm c}\sim 1.5-2.5 R_{\rm 200c}$. Our results can be used to constrain theoretical models, particularly the physics of supernova and AGN feedback, as well as their interplay with environmental processes, through comparison with current and future X-ray and SZ observations.

</div>

<div id="div_fig1">

<img src="tmp_2211.07659/./Figs/R_Closure_TNG_EAGLE_SIMBA.png" alt="Fig5.1" width="50%"/><img src="tmp_2211.07659/./Figs/baryonFrac_2dhist_TNG_EAGLE_SIMBA.png" alt="Fig5.2" width="50%"/>

**Figure 5. -** Top panel: The characteristic scale i.e. "closure radius" $R_{\rm c}$ within which all baryons associated with dark matter are found in TNG, EAGLE, and SIMBA at $z=0$. We show the median value as a function of halo $M_{\rm 200c}$ over all haloes at each given $M_{\rm 200c}$ bin. The shaded regions correspond to 16\% and 84\% percentile halo-to-halo variation. The closure radius is a strong function of the halo mass with different amplitude for each simulation. Bottom row: 2D histograms of the average ‘cumulative’ baryon fraction (colours) as a function of the halo mass (y-axis) and halocentric distance (x-axis) in TNG, EAGLE, and SIMBA. The orange bright regions show regimes where baryons are missing, whereas the dark regions show regions at the cosmic baryon fraction.
     (*Fig: R_c*)

</div>
<div id="div_fig2">

<img src="tmp_2211.07659/./Figs/R_Closure_TNG_VAR.png" alt="Fig6.1" width="20%"/><img src="tmp_2211.07659/./Figs/baryonFrac_2dhist_cum_TNG_VAR0000_z0.png" alt="Fig6.2" width="20%"/><img src="tmp_2211.07659/./Figs/baryonFrac_2dhist_cum_TNG_VAR2101_z0.png" alt="Fig6.3" width="20%"/><img src="tmp_2211.07659/./Figs/baryonFrac_2dhist_cum_TNG_VAR2201_z0.png" alt="Fig6.4" width="20%"/><img src="tmp_2211.07659/./Figs/baryonFrac_2dhist_cum_TNG_VAR0030_z0.png" alt="Fig6.5" width="20%"/>

**Figure 6. -** Top row: The characteristic scale i.e. "closure radius" $R_{\rm c}$ within which all baryons associated with dark matter are found, for several variants of the TNG model performed in $\sim \rm 37 Mpc$ boxes (see text). We show the median value as a function of halo $M_{\rm 200c}$ over all haloes at each given $M_{\rm 200c}$ bin. The shaded regions correspond to 16\% and 84\% percentiles. The solid black line corresponds to our expectation of the non-radiative run at a sufficiently high-resolution, while the dashed line is the actual resolution-impacted result at the resolution of the other depicted runs (see Appendix \ref{app: TNG_convergence}). Middle and bottom rows: 2D histograms of the average ‘cumulative’ baryon fraction (colours) as a function of the halo mass (y-axis) and halocentric distance (x-axis) in four TNG model variants. The orange and green colours correspond to the cumulative baryon fraction less and greater than the cosmic value, respectively. We can see that $R_{\rm c}>R_{\rm 200c}$, i.e. the excess of baryons beyond $R_{\rm 200c}$, is explained by the influence of astrophysical feedback processes, each of which impacts $R_{\rm c}$ at different halo mass scales. (*Fig: TNG_VAR_R_c*)

</div>
<div id="div_fig3">

<img src="tmp_2211.07659/./Figs/R_Closure_fit_z0.png" alt="Fig10.1" width="50%"/><img src="tmp_2211.07659/./Figs/R_Closure_fit_beta.png" alt="Fig10.2" width="50%"/>

**Figure 10. -** Correlation between the halo baryon fraction and the closure radius $R_{c}$. Left: Normalised closure radius as a function of the halo baryon fraction within $R_{\rm 200c}$ at three different redshifts. Our universal fitting formula, $R_{\rm c}/R_{\rm 200c,500c} - 1 = \beta(z) [1 - f_{\rm b}(<R_{\rm 200c,500c})/f_{\rm b,cosmic}]$, is valid for groups and clusters with $\log_{10}(M_{\rm 200c}/{\rm M_{\odot}})\gtrsim 13$ of all simulations considered: TNG, EAGLE, and SIMBA. The solid lines show the best fit for each simulation, and the shaded regions correspond to the $1\sigma$ of the scatter around the best fit. Replacing $M_{\rm 200c}$ and $R_{\rm 200c}$ with $M_{\rm 500c}$ and $R_{\rm 500c}$ would only alter the slope of our best fit (see Table \ref{tab: universal_r_c}), but leaves the shape of the fitting function and our conclusions unchanged. Right: $\beta(z)$ in our fitting formula (Eqs. \ref{eq: universal_r_c},\ref{eq: beta}) as a function of redshift. The curved lines show our best fit for $\beta(z)$ using the parameters in Table \ref{tab: universal_r_c}. The dots correspond to best values for $\beta(z)$ when fitting Eq. \ref{eq: universal_r_c} at each individual redshift. (*Fig: universal_relation_r_c*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

#  3D Map of Interstellar Dust Temperature

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.07667-b31b1b.svg)](https://arxiv.org/abs/2211.07667)<mark>Appeared on: 2022-11-16</mark> - _11 Figures, 19 pages_

</div>
<div id="authors">

Ioana A. Zelko, Douglas P. Finkbeiner, <mark>Albert Lee</mark>, <mark>Gregory Green</mark>

</div>
<div id="abstract">

**Abstract:** We present the first large-scale 3D map of interstellar dust temperature. We build upon existing 3D reddening maps derived from starlight absorption (Bayestar19), covering 3/4 of the sky. Starting with the column density for each of 500 million 3D voxels, we propose a temperature and emissivity power-law slope ($\beta$) for each of them, and integrate along the line of sight to synthesize an emission map in five frequency bands observed by\emph{Planck}and\emph{IRAS}.  The reconstructed emission map is constrained to match observations on a$10'$scale, and does so with good fidelity. We produce 3D temperature maps at  resolutions of$110', 55', $and$27'$. We assess performance on Cepheus, a dust cloud with two distinct components along the line of sight, and find distinct temperatures for the two components. We thus show that this methodology has enough precision to constrain clouds with different temperature along the line of sight up to$1-\sigma$error. This would be an important result for dust frequency decorrelation foreground analysis for cosmic microwave background experiments, which would be impacted by a line-of-sight with varying temperature and magnetic field components. In addition to$T$and$\beta$, we constrain the conversion factor between emission optical depth and reddening. This conversion factor is assumed to be constant in commonly used emission-based reddening maps. However, this work shows a factor of two variation that may prove significant for some applications.

</div>

<div id="div_fig1">

<img src="tmp_2211.07667/./figures/smoothing/planck_545.jpg" alt="Fig5.1" width="25%"/><img src="tmp_2211.07667/./figures/smoothing/total_zoom_in_reconstructed_emisssion_545.jpg" alt="Fig5.2" width="25%"/><img src="tmp_2211.07667/./figures/smoothing/planck_zoom_500_545_smooth_1024.jpg" alt="Fig5.3" width="25%"/><img src="tmp_2211.07667/./figures/smoothing/planck_zoom_500_545_smooth_total_reconstructed_emission_1024.jpg" alt="Fig5.4" width="25%"/>

**Figure 5. -** Smoothed Maps(a) The data for dust emission at 545 GHz, after the processing described in Section \S\ref{sec:dust_3D_data}. The plot is zoomed in on the sky area of the $\rho$ Ophiuchi cloud complex. (b) The reconstruction of the emission data in (a) using the 3D dust reddening fits. We noticed that even though the images match well in general, the reconstruction has higher resolution than the emission data. This is due to Bayestar having higher resolution than the \emph{Planck} maps. Thus, smoothing is required to match the "point spread functions" of the emission and reddening data. (c) Data emission map after smoothing to 10'. (d) Reconstructed emission map based on both emission data and Bayestar smoothed to 10'. Figure \ref{fig:difference_smoothed_maps} shows the difference between maps (c) and (d). (*fig:smoothed_maps*)

</div>
<div id="div_fig2">

<img src="tmp_2211.07667/./figures/smoothing/planck_zoom_500_545_smooth_total_difference_emission_1024.jpg" alt="Fig1" width="100%"/>

**Figure 1. -** Difference Between Smoothed Data and Reconstructed MapsThe difference between the smoothed emission data and the reconstruction based on the smoothed reddening map ( Figs (c) and (d) in \ref{fig:smoothed_maps}. If the relative point spread functions would not match, we would see contours appear around brighter spots; the fact that we don't see them is an indication the smoothing was successful. (*fig:difference_smoothed_maps*)

</div>
<div id="div_fig3">

<img src="tmp_2211.07667/./figures/planck_2998.png" alt="Fig3.1" width="20%"/><img src="tmp_2211.07667/./figures/planck_217.png" alt="Fig3.2" width="20%"/><img src="tmp_2211.07667/./figures/planck_353.png" alt="Fig3.3" width="20%"/><img src="tmp_2211.07667/./figures/planck_545.png" alt="Fig3.4" width="20%"/><img src="tmp_2211.07667/./figures/planck_857.png" alt="Fig3.5" width="20%"/>

**Figure 3. -** Sky Dust Emission MapsThe sky-wide dust emission maps. The are obtained from the 217, 353, 545, 857GHz emission maps from the \emph{Planck}  satellite  and the 2998GHz \emph{IRAS/DIRBE} map, after subtracting the contribution of the CMB. The other foregrounds are subdominant to dust at these frequencies, with the exception of the CIB. This analysis thus applies only to the lines of sight where the contribution from the ISM dust is big enough to dominate the CIB. (*fig:dust_3D_emissison_data*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\todo}[1]{\textcolor{red}{#1}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\todo$}[1]{\textcolor{red}{#1}}$</div>



<div id="title">

# Radial Velocity Survey for Planets around Young stars (RVSPY): Based on observations collected at the European Organization for Astronomical Research in the Southern Hemisphere under MPG programs 0101.A-9012, 0103.A-9010, 0104.A-9003, 0107.A-9004, 0108.A-9014, 0109.A-9014 and ESO programs 098.C-0739, 1101.C-0557.

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.08294-b31b1b.svg)](https://arxiv.org/abs/2211.08294)<mark>Appeared on: 2022-11-16</mark> - _10 pages, 9 figures, 5 tables; Accepted for publication in A&A Letters_

</div>
<div id="authors">

<mark><mark>O. Zakhozhay</mark></mark>, et al. -- incl., <mark><mark>R. Launhardt</mark></mark>, <mark><mark>T. Trifonov</mark></mark>, <mark><mark>M. Kürster</mark></mark>, <mark><mark>Th. Henning</mark></mark>

</div>
<div id="abstract">

**Abstract:** Aiming to detect planetary companions to young stars with debris disks via the radial velocity method, we observed HD114082 during April 2018 - August 2022 as one of the targets of our RVSPY program (Radial Velocity Survey for Planets around Young stars). We used the FEROS spectrograph, mounted to the MPG/ESO 2.2 m telescope in Chile, to obtain high signal-to-noise spectra and time series of precise radial velocities (RVs). Additionally, we analyzed archival HARPS spectra and TESS photometric data. We used the CERES, CERES++ and SERVAL pipelines to derive RVs and activity indicators and ExoStriker for the independent and combined analysis of the RVs and TESS photometry. We report the discovery of a warm super-Jovian companion around HD114082 based on a 109.8$\pm$0.4 day signal in the combined RV data from FEROS and HARPS, and on one transit event in the TESS photometry. The best-fit model indicates a 8.0$\pm$1.0 Mjup companion with a radius of 1.00$\pm$0.03 Rjup in an orbit with a semi-major axis of 0.51$\pm$0.01 au and an eccentricity of 0.4$\pm$0.04. The companions orbit is in agreement with the known near edge-on debris disk located at about 28 au. HD114082b is possibly the youngest (15$\pm$6 Myr), and one of only three younger than 100 Myr giant planetary companions for which both their mass and radius have been determined observationally. It is probably the first properly model-constraining giant planet that allows distinguishing between hot and cold-start models. It is significantly more compatible with the cold-start model. 

</div>

<div id="div_fig1">

<img src="tmp_2211.08294/./RV_timeSeries_tr.png" alt="Fig7" width="100%"/>

**Figure 7. -** \label{fig:RV}
RV time series data for HD 114082. FEROS data from RVSPY are shown as black circles. The RVs computed from the publicly available HARPS spectra are shown as gray circles. Error bars account for the suspected activity jitter (see text for details). The solid gray line shows the best-fit model (see parameters in Table \ref{tab:plan_pars}), and the vertical blue line indicates the time of the observed transit. The lower panel shows the residuals of the fit.
 (*fig:RV*)

</div>
<div id="div_fig2">

<img src="tmp_2211.08294/./FEROS_correlations.png" alt="Fig8" width="100%"/>

**Figure 8. -** \label{fig:activity_feros}
Time series and GLS of the RVs and activity indicators derived from the FEROS spectra (left and middle panels, respectively). The right panels show the correlation with respect to the FEROS RVs for the derived key activity indicators. The type of activity indicator is given in each of the panels of the middle column. The vertical red line shows the suspected planetary period ($P$ = 109.8 d), from the model with linear trend of the combined RVs (FEROS+HARPS) and photometric (TESS) data.
 (*fig:activity_feros*)

</div>
<div id="div_fig3">

<img src="tmp_2211.08294/./transit.png" alt="Fig3" width="100%"/>

**Figure 3. -** \label{fig:TESS}
Baseline-corrected sector 38 {\it TESS} data (relative flux) for HD 114082 around the transit event with the best-fit model of the transit overlaid (black line). The bottom panel shows the residuals of the fit.
 (*fig:TESS*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

91  publications in the last 7 days.
	 _build/html/2211.08294.md
	 _build/html/2211.07667.md
	 _build/html/2211.07659.md
	 _build/html/2211.06454.md
	 _build/html/2211.06205.md
	 _build/html/2211.06194.md
	 _build/html/2211.05022.md
	 _build/html/2211.04048.md
	 _build/html/2211.03641.md
	 _build/html/2211.02502.md
	 _build/html/2211.01474.md
	 _build/html/2211.00667.md
	 _build/html/2211.00657.md
	 _build/html/2211.00087.md
	 _build/html/2211.00036.md
	 _build/html/2211.00020.md
	 _build/html/2211.00010.md
	 _build/html/2210.17434.md
	 _build/html/2210.12412.md
	 _build/html/2210.11926.md
	 _build/html/2210.11551.md
	 _build/html/2210.11103.md
	 _build/html/2210.10707.md
	 _build/html/2210.09930.md
	 _build/html/2210.08996.md
	 _build/html/2210.08517.md
	 _build/html/2210.08510.md
	 _build/html/2210.08078.md
	 _build/html/2210.06504.md
	 _build/html/2210.05695.md
	 _build/html/2210.05595.md
	 _build/html/2210.05505.md
	 _build/html/2210.04721.md
	 _build/html/2210.03615.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers